# Tutorial: Creating an AnnData Object from Tahoe-100M Dataset
https://hf-mirror.com/datasets/tahoebio/Tahoe-100M/blob/main/tutorials/loading_data.ipynb

https://hf-mirror.com/datasets/tahoebio/Tahoe-100M

This notebook is intented for users who are familiar with the anndata format for single-cell data. 

We'll walk through how to parse records in the huggingface dataset format and convert between the two.

## Install Required Libraries

```
!pip install datasets anndata scipy pandas pubchempy
```

## Import Libraries

In [1]:
from datasets import load_dataset
from scipy.sparse import csr_matrix
from tqdm import tqdm
import anndata
import pandas as pd
import pubchempy as pcp
import numpy as np
import ast

c:\Users\Cavendishes\.conda\envs\tahoe-cvae\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Mapping records to anndata

This function takes in a generator that emits records from the Tahoe-100M huggingface dataset and returns an anndata object. 

Use the `sample_size` argument to specify the number of records you need. 

You can also create a new generator using the `dataset.filter` function to only emit records that match a certain filter (eg: for a specific drug/plate/sample).

If you'd like to create a DataLoader for an ML training application, it's likely best to use the data in it's native format without interfacing with anndata.

In [2]:
def create_anndata_from_generator(generator, gene_vocab, sample_size=None):
    sorted_vocab_items = sorted(gene_vocab.items())
    token_ids, gene_names = zip(*sorted_vocab_items)
    token_id_to_col_idx = {token_id: idx for idx, token_id in enumerate(token_ids)}

    data, indices, indptr = [], [], [0]
    obs_data = []

    for i, cell in tqdm(enumerate(generator), total = sample_size):
        if sample_size is not None and i >= sample_size:
            break
        genes = cell['genes']
        expressions = cell['expressions']
        if expressions[0] < 0:
            genes = genes[1:]
            expressions = expressions[1:]

        col_indices = [token_id_to_col_idx[gene] for gene in genes if gene in token_id_to_col_idx]
        valid_expressions = [expr for gene, expr in zip(genes, expressions) if gene in token_id_to_col_idx]

        data.extend(valid_expressions)
        indices.extend(col_indices)
        indptr.append(len(data))

        obs_entry = {k: v for k, v in cell.items() if k not in ['genes', 'expressions']}
        obs_data.append(obs_entry)

    expr_matrix = csr_matrix((data, indices, indptr), shape=(len(indptr) - 1, len(gene_names)))
    obs_df = pd.DataFrame(obs_data)

    adata = anndata.AnnData(X=expr_matrix, obs=obs_df)
    adata.var.index = pd.Index(gene_names, name='ensembl_id')

    return adata


## Load Tahoe-100M Dataset

In [3]:
# setup dataset
tahoe_100m_ds = load_dataset('vevotx/Tahoe-100M', streaming=True, split='train')

## Load Gene Metadata

The gene metadata contains the mapping between the integer token IDs used in the dataset and standard identifiers for genes (ensembl IDs and HGNC gene symbols)

基因元数据包含数据集中使用的整数标记 ID 与基因标准标识符（集成 ID 和 HGNC 基因符号）之间的映射

In [4]:
gene_metadata = load_dataset("vevotx/Tahoe-100M", name="gene_metadata", split="train")
gene_vocab = {entry["token_id"]: entry["ensembl_id"] for entry in gene_metadata}

In [5]:
import json
# 将 gene_vocab 保存下来
with open("gene_vocab.json", "w") as f:
    json.dump(gene_vocab, f)

In [ ]:
# 加载 gene_vocab
with open("gene_vocab.json", "r") as f:
    gene_vocab = json.load(f)

## Create AnnData Object

In [5]:
adata = create_anndata_from_generator(tahoe_100m_ds, gene_vocab, sample_size=500000) # 1000 000

100%|██████████| 500000/500000 [11:43<00:00, 710.43it/s] 
c:\Users\Cavendishes\.conda\envs\tahoe-cvae\Lib\functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


In [6]:
adata

AnnData object with n_obs × n_vars = 500000 × 62710
    obs: 'drug', 'sample', 'BARCODE_SUB_LIB_ID', 'cell_line_id', 'moa-fine', 'canonical_smiles', 'pubchem_cid', 'plate'

## Inspect Metadata (`adata.obs`)

In [7]:
adata.obs.head()

,drug,sample,BARCODE_SUB_LIB_ID,cell_line_id,moa-fine,canonical_smiles,pubchem_cid,plate
0,8-Hydroxyquinoline,smp_1783,01_001_052-lib_1105,CVCL_0480,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4
1,8-Hydroxyquinoline,smp_1783,01_001_105-lib_1105,CVCL_0546,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4
2,8-Hydroxyquinoline,smp_1783,01_001_165-lib_1105,CVCL_1717,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4
3,8-Hydroxyquinoline,smp_1783,01_003_094-lib_1105,CVCL_1717,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4
4,8-Hydroxyquinoline,smp_1783,01_003_164-lib_1105,CVCL_1056,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4


## Enrich with Sample Metadata

Although the main data contains several metadata fields, there are some additional columns (such as drug concentration) which are omitted to reduce the size of the data. 

If they are needed, they may be fetched using the sample_metadata.

虽然主数据包含多个元数据字段，但省略了一些额外的列（例如药物浓度）以减小数据的大小。如果需要，可以使用 sample_metadata 获取它们。

In [8]:
sample_metadata = load_dataset("vevotx/Tahoe-100M","sample_metadata", split="train").to_pandas()
adata.obs = pd.merge(adata.obs, sample_metadata.drop(columns=["drug","plate"]), on="sample")
adata.obs.head()

c:\Users\Cavendishes\.conda\envs\tahoe-cvae\Lib\functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


,drug,sample,BARCODE_SUB_LIB_ID,cell_line_id,moa-fine,canonical_smiles,pubchem_cid,plate,mean_gene_count,mean_tscp_count,mean_mread_count,mean_pcnt_mito,drugname_drugconc
0,8-Hydroxyquinoline,smp_1783,01_001_052-lib_1105,CVCL_0480,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,2738.463797,0.023783,"[('8-Hydroxyquinoline', 0.05, 'uM')]"
1,8-Hydroxyquinoline,smp_1783,01_001_105-lib_1105,CVCL_0546,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,2738.463797,0.023783,"[('8-Hydroxyquinoline', 0.05, 'uM')]"
2,8-Hydroxyquinoline,smp_1783,01_001_165-lib_1105,CVCL_1717,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,2738.463797,0.023783,"[('8-Hydroxyquinoline', 0.05, 'uM')]"
3,8-Hydroxyquinoline,smp_1783,01_003_094-lib_1105,CVCL_1717,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,2738.463797,0.023783,"[('8-Hydroxyquinoline', 0.05, 'uM')]"
4,8-Hydroxyquinoline,smp_1783,01_003_164-lib_1105,CVCL_1056,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,2738.463797,0.023783,"[('8-Hydroxyquinoline', 0.05, 'uM')]"


### Extract drug concentration from drugname_drugconc

In [9]:
# convert drugname_drugconc to drug_name, drug_conc, drug_unit

adata.obs["drugname_drugconc"] = adata.obs["drugname_drugconc"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)


In [10]:

# extact drug concentration name faster

# Convert column to NumPy array of tuples
vals = adata.obs["drugname_drugconc"].values

# Extract tuples efficiently
drug_array = np.array([x[0] if isinstance(x, list) and len(x) > 0 else (None, None, None) for x in vals])


In [11]:

# add drug array values
adata.obs["drug_name"] = drug_array[:, 0]
adata.obs["drug_conc"] = drug_array[:, 1]
adata.obs["drug_unit"] = drug_array[:, 2]


## Add Drug Metadata

The drug metadata contains additional information for the compounds used in Tahoe-100M. 

See the dataset card and our [paper](https://www.biorxiv.org/content/10.1101/2025.02.20.639398v1) for more information about how this information was generated.

药物元数据包含 Tahoe-100M 中使用的化合物的附加信息。有关如何生成此信息的更多信息，请参阅数据集卡和我们的论文 

In [ ]:

# get drug metadata
drug_metadata = load_dataset("vevotx/Tahoe-100M","drug_metadata", split="train").to_pandas()

adata.obs = pd.merge(adata.obs, drug_metadata.drop(columns=["canonical_smiles","pubchem_cid","moa-fine"]), on="drug", how='left')

adata.obs.head()


c:\Users\Cavendishes\.conda\envs\tahoe-cvae\Lib\functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


,drug,sample,BARCODE_SUB_LIB_ID,cell_line_id,moa-fine,canonical_smiles,pubchem_cid,plate,mean_gene_count,mean_tscp_count,...,mean_pcnt_mito,drugname_drugconc,drug_name,drug_conc,drug_unit,targets,moa-broad,human-approved,clinical-trials,gpt-notes-approval
0,8-Hydroxyquinoline,smp_1783,01_001_052-lib_1105,CVCL_0480,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,...,0.023783,"[(8-Hydroxyquinoline, 0.05, uM)]",8-Hydroxyquinoline,0.05,uM,None,unclear,no,yes,"Used in some clinical trial formulations, not ..."
1,8-Hydroxyquinoline,smp_1783,01_001_105-lib_1105,CVCL_0546,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,...,0.023783,"[(8-Hydroxyquinoline, 0.05, uM)]",8-Hydroxyquinoline,0.05,uM,None,unclear,no,yes,"Used in some clinical trial formulations, not ..."
2,8-Hydroxyquinoline,smp_1783,01_001_165-lib_1105,CVCL_1717,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,...,0.023783,"[(8-Hydroxyquinoline, 0.05, uM)]",8-Hydroxyquinoline,0.05,uM,None,unclear,no,yes,"Used in some clinical trial formulations, not ..."
3,8-Hydroxyquinoline,smp_1783,01_003_094-lib_1105,CVCL_1717,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,...,0.023783,"[(8-Hydroxyquinoline, 0.05, uM)]",8-Hydroxyquinoline,0.05,uM,None,unclear,no,yes,"Used in some clinical trial formulations, not ..."
4,8-Hydroxyquinoline,smp_1783,01_003_164-lib_1105,CVCL_1056,unclear,C1=CC2=C(C(=C1)O)N=CC=C2,1923.0,plate4,1478.268171,2341.339094,...,0.023783,"[(8-Hydroxyquinoline, 0.05, uM)]",8-Hydroxyquinoline,0.05,uM,None,unclear,no,yes,"Used in some clinical trial formulations, not ..."


## Drug Info from PubChem

We also provide the pubchem IDs for the compounds in Tahoe, this can be used to querry additional information as needed.

我们还提供 Tahoe 化合物的 pubchem ID，这可用于根据需要查询更多信息。

In [13]:
drug_name = adata.obs["drug"].values[0]
cid = int(float(adata.obs["pubchem_cid"].values[0]))
compound = pcp.Compound.from_cid(cid)

print(f"Name: {drug_name}")
print(f"Synonyms: {compound.synonyms[:10]}")
print(f"Formula: {compound.molecular_formula}")
print(f"SMILES: {compound.isomeric_smiles}")
print(f"Mass: {compound.exact_mass}")

Name: 8-Hydroxyquinoline
Synonyms: ['8-HYDROXYQUINOLINE', 'quinolin-8-ol', '148-24-3', '8-quinolinol', 'Oxyquinoline', 'Oxine', 'Quinophenol', 'Oxychinolin', '8-Quinol', '8-Oxyquinoline']
Formula: C9H7NO
SMILES: None
Mass: 145.052763847


## Load Cell Line Metadata
The cell-line metadata contains additional identifiers for the
cell-lines used in Tahoe (eg: Depmap-IDs) as well as a curated list of driver mutations for each cell line. This information can be used for instance to train genotype aware models on the Tahoe data.

细胞系元数据包含 Tahoe 中使用的细胞系的其他标识符（例如：Depmap-ID）以及每个细胞系的驱动突变的精选列表。例如，此信息可用于在 Tahoe 数据上训练基因型感知模型。

In [14]:

cell_line_metadata = load_dataset("vevotx/Tahoe-100M","cell_line_metadata", split="train").to_pandas()

# save cell line to csv
# cell_line_metadata.to_csv("/home/ubuntu/anatoly-tahoe-100-texas/data/tahoe-100m_5M_cell_line_metadata.csv", index=False)
cell_line_metadata.to_csv("../data/tahoe-100m_5M_cell_line_metadata.csv", index=False)

# show cell names
cell_line_metadata.head(5)


,cell_name,Cell_ID_DepMap,Cell_ID_Cellosaur,Organ,Driver_Gene_Symbol,Driver_VarZyg,Driver_VarType,Driver_ProtEffect_or_CdnaEffect,Driver_Mech_InferDM,Driver_GeneType_DM
0,A549,ACH-000681,CVCL_0023,Lung,CDKN2A,Hom,Deletion,DEL,LoF,Suppressor
1,A549,ACH-000681,CVCL_0023,Lung,CDKN2B,Hom,Deletion,DEL,LoF,Suppressor
2,A549,ACH-000681,CVCL_0023,Lung,KRAS,Hom,Missense,p.G12S,GoF,Oncogene
3,A549,ACH-000681,CVCL_0023,Lung,SMARCA4,Hom,Frameshift,p.Q729fs,LoF,Suppressor
4,A549,ACH-000681,CVCL_0023,Lung,STK11,Hom,Stopgain,p.Q37*,LoF,Suppressor


### Save the data

In [15]:

 # convert targets to string
adata.obs["targets"] = adata.obs["targets"].astype(str)

# rename drug_conc as string
adata.obs["drugname_drugconc"] = adata.obs["drugname_drugconc"].astype(str)


In [16]:
# save down-sampled adata file
adata.write_h5ad("../data/tahoe-100m_5M.h5ad")
